In [ ]:
# 0. Install dependancies and data

In [7]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import os

In [5]:
tf.__version__

'2.8.0'

In [10]:
# Reading in our data
df = pd.read_csv(
    os.path.join('dataset','train.csv','train.csv')
)

In [11]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [12]:
df.tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0
159570,fff46fc426af1f9a,"""\nAnd ... I really don't think you understand...",0,0,0,0,0,0


In [ ]:
# 1. Preprocessing

In [13]:
from tensorflow.keras.layers import TextVectorization
# used for tokenisation

In [14]:
X = df['comment_text']
y = df[df.columns[2:]].values

In [15]:
X.head()

0    Explanation\nWhy the edits made under my usern...
1    D'aww! He matches this background colour I'm s...
2    Hey man, I'm really not trying to edit war. It...
3    "\nMore\nI can't make any real suggestions on ...
4    You, sir, are my hero. Any chance you remember...
Name: comment_text, dtype: object

In [17]:
print(y)

[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 ...
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]


In [18]:
MAX_FEATURES = 20000 # number of words in the vocab. More words equal a bigger model( more training time)

In [19]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,output_sequence_length=1800,output_mode='int')
# output_sequence_length = length/capacity of sentence
# output_mode = what datatype to convert the word to 

In [20]:
vectorizer.adapt(X.values) # learn our vocab, basically maps our words to numbers

In [23]:
vectorizer('I hate you, okay')[:2] #example

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([  8, 363])>

In [24]:
vectorized_text = vectorizer(X.values)

In [25]:
vectorized_text # tokenised

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[ 645,   76,    2, ...,    0,    0,    0],
       [   1,   54, 2489, ...,    0,    0,    0],
       [ 425,  441,   70, ...,    0,    0,    0],
       ...,
       [   1, 7392,  383, ...,    0,    0,    0],
       [   5,   12,  534, ...,    0,    0,    0],
       [   5,    8,  130, ...,    0,    0,    0]])>

In [26]:
# data pipeline, makes it easy to work with large data . Almost like combining the input and label
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text,y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # helps bottlenecks

In [28]:
batch_X, batch_y = dataset.as_numpy_iterator().next() # input and label

In [30]:
# Basically we are creating batches. The model will go through each batch and perform feed forward and backprop
train = dataset.take(int(len(dataset)*.7)) # basically taking 70% of the data (in batches)
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2)) # skip the first 70% and take 20%
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1)) # skip the first 90% and take 10%

In [ ]:
# 2. Create Sequential Model